<a href="https://colab.research.google.com/github/AndrewstheBuilder/sms_spam_detection/blob/main/SMS_SPAM_DETECTION_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import auth
auth.authenticate_user()

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [30]:
from google.colab import files

uploaded = files.upload()

Saving spam.csv to spam (1).csv


In [7]:
!wget http://nlp.stanford.edu/data/glove.6B.zip
!unzip glove.6B.zip

--2024-03-06 20:46:54--  http://nlp.stanford.edu/data/glove.6B.zip
Resolving nlp.stanford.edu (nlp.stanford.edu)... 171.64.67.140
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:80... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://nlp.stanford.edu/data/glove.6B.zip [following]
--2024-03-06 20:46:54--  https://nlp.stanford.edu/data/glove.6B.zip
Connecting to nlp.stanford.edu (nlp.stanford.edu)|171.64.67.140|:443... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip [following]
--2024-03-06 20:46:55--  https://downloads.cs.stanford.edu/nlp/data/glove.6B.zip
Resolving downloads.cs.stanford.edu (downloads.cs.stanford.edu)... 171.64.64.22
Connecting to downloads.cs.stanford.edu (downloads.cs.stanford.edu)|171.64.64.22|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 862182613 (822M) [application/zip]
Saving to: ‘glove.6B.zip’

glov

In [9]:
import gensim.downloader as api
embeddings = api.load("glove-wiki-gigaword-50")

[==================================================] 100.0% 66.0/66.0MB downloaded


In [10]:
embeddings['hello']

array([-0.38497 ,  0.80092 ,  0.064106, -0.28355 , -0.026759, -0.34532 ,
       -0.64253 , -0.11729 , -0.33257 ,  0.55243 , -0.087813,  0.9035  ,
        0.47102 ,  0.56657 ,  0.6985  , -0.35229 , -0.86542 ,  0.90573 ,
        0.03576 , -0.071705, -0.12327 ,  0.54923 ,  0.47005 ,  0.35572 ,
        1.2611  , -0.67581 , -0.94983 ,  0.68666 ,  0.3871  , -1.3492  ,
        0.63512 ,  0.46416 , -0.48814 ,  0.83827 , -0.9246  , -0.33722 ,
        0.53741 , -1.0616  , -0.081403, -0.67111 ,  0.30923 , -0.3923  ,
       -0.55002 , -0.68827 ,  0.58049 , -0.11626 ,  0.013139, -0.57654 ,
        0.048833,  0.67204 ], dtype=float32)

In [34]:
# Start LSTM Model for Spam Detection
# LSTMs are the industry standard model to counter vanishing/exploding gradients
import numpy as np
import tensorflow
np.random.seed(0)
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, Input, Dropout, LSTM, Activation
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing import sequence
from tensorflow.keras.initializers import glorot_uniform
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from keras.utils import to_categorical

np.random.seed(1)

In [38]:
# Utils
# UNQ_C3 (UNIQUE CELL IDENTIFIER, DO NOT EDIT)
# GRADED FUNCTION: sentences_to_indices

def sentences_to_indices(X, word_to_index, max_len):
    """
    Converts an array of sentences (strings) into an array of indices corresponding to words in the sentences.
    The output shape should be such that it can be given to `Embedding()` (described in Figure 4).

    Arguments:
    X -- array of sentences (strings), of shape (m,)
    word_to_index -- a dictionary containing the each word mapped to its index
    max_len -- maximum number of words in a sentence. You can assume every sentence in X is no longer than this.

    Returns:
    X_indices -- array of indices corresponding to words in the sentences from X, of shape (m, max_len)
    """

    m = X.shape[0]                                   # number of training examples

    ### START CODE HERE ###
    # Initialize X_indices as a numpy matrix of zeros and the correct shape (≈ 1 line)
    X_indices = np.zeros((m,max_len))

    for i in range(m):                               # loop over training examples

        # Convert the ith training sentence to lower case and split it into words. You should get a list of words.
        sentence_words = X[i].lower().split()

        # Initialize j to 0
        j = 0

        # Loop over the words of sentence_words

        for w in sentence_words:
            # if w exists in the word_to_index dictionary
            if w in word_to_index:
                # Set the (i,j)th entry of X_indices to the index of the correct word.
                X_indices[i, j] = word_to_index[w]
                # Increment j to j + 1
                j =  j+1

    ### END CODE HERE ###

    return X_indices

def pretrained_embedding_layer(word_to_vec_map, word_to_index):
    """
    Creates a Keras Embedding() layer and loads in pre-trained GloVe 50-dimensional vectors.

    Arguments:
    word_to_vec_map -- dictionary mapping words to their GloVe vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary

    Returns:
    embedding_layer -- pretrained layer Keras instance
    """

    vocab_size = len(word_to_index) + 1
    any_word = next(iter(word_to_index.keys()))
    emb_dim = word_to_vec_map[any_word].shape[0]

     # Step 1
    # Initialize the embedding matrix as a numpy array of zeros.
    emb_matrix = np.zeros((vocab_size, emb_dim))

    # Step 2
    # Set each row "idx" of the embedding matrix to be
    # the word vector representation of the idx'th word of the vocabulary
    for word, idx in word_to_index.items():
        emb_matrix[idx, :] = word_to_vec_map[word]

    # Step 3
    # Define Keras embedding layer with the correct input and output sizes
    # Make it non-trainable because we are just loading in pretrained GloVe 50-dimensional vectors
    embedding_layer = Embedding(input_dim=vocab_size , output_dim=emb_dim , trainable=False)

    # Step 4 (already done for you; please do not modify)
    # Build the embedding layer, it is required before setting the weights of the embedding layer.
    embedding_layer.build((None,))


    # Set the weights of the embedding layer to the embedding matrix. Your layer is now pretrained.
    embedding_layer.set_weights([emb_matrix])

    return embedding_layer

# Functions taken from emo_utils.py
def read_glove_vecs(embeddings):
      # for line in embeddings.vocab.keys():
      #     print('line',line)
      #     line = line.strip().split()
      #     curr_word = line[0]
      #     words.add(curr_word)
      #     word_to_vec_map[curr_word] = np.array(embeddings[curr_word], dtype=np.float64)

      words_to_index = embeddings.key_to_index
      index_to_words = embeddings.index_to_key
      word_to_vec_map = embeddings
      return words_to_index, index_to_words, word_to_vec_map

def convert_to_one_hot(Y, C):
    print('Y.reshape(-1)',Y.reshape(-1))
    print('C',C)
    Y = np.eye(C)[Y.reshape(-1)]
    return Y

def read_csv(filename = 'data/emojify_data.csv'):
    phrase = []
    emoji = []

    with open (filename) as csvDataFile:
        csvReader = csv.reader(csvDataFile)

        for row in csvReader:
            phrase.append(row[0])
            emoji.append(row[1])

    X = np.asarray(phrase)
    Y = np.asarray(emoji, dtype=int)

    return X[:10], Y[:10]

In [28]:
 word_to_index, index_to_word, word_to_vec_map = read_glove_vecs(embeddings)

In [33]:
X1 = np.array(["funny lol", "lets play baseball", "food is ready for you"])
X1_indices = sentences_to_indices(X1, word_to_index, max_len=5)
print("X1 =", X1)
print("X1_indices =\n", X1_indices)

# Load in csv dataset
df = pd.read_csv('spam.csv',delimiter=',',encoding='latin-1')
df.head()
df.info()

df.drop(['Unnamed: 2', 'Unnamed: 3', 'Unnamed: 4'],axis=1,inplace=True)
df.info()

# Get input and outputs
X = df.v2.iloc[:100]
Y = df.v1.iloc[:100]
X_train,X_test,Y_train,Y_test = train_test_split(X,Y,test_size=0.20)
#,Y_train,Y_test =
# print(len(X_train))
# print(len(Y_train))
# print(len(Y_test))
# print(len(Y_test)/(len(Y_train)+len(Y_test)))

maxLen = len(max(X_train, key=lambda x: len(x.split())).split())
maxLen

# word_to_index, index_to_word, word_to_vec_map = read_glove_vecs('data/glove.6B.50d.txt')

X1 = ['funny lol' 'lets play baseball' 'food is ready for you']
X1_indices =
 [[5.4660e+03 7.3048e+04 0.0000e+00 0.0000e+00 0.0000e+00]
 [8.2350e+03 2.8200e+02 1.4440e+03 0.0000e+00 0.0000e+00]
 [5.6500e+02 1.4000e+01 1.1880e+03 1.0000e+01 8.1000e+01]]
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   v1          5572 non-null   object
 1   v2          5572 non-null   object
 2   Unnamed: 2  50 non-null     object
 3   Unnamed: 3  12 non-null     object
 4   Unnamed: 4  6 non-null      object
dtypes: object(5)
memory usage: 217.8+ KB
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5572 entries, 0 to 5571
Data columns (total 2 columns):
 #   Column  Non-Null Count  Dtype 
---  ------  --------------  ----- 
 0   v1      5572 non-null   object
 1   v2      5572 non-null   object
dtypes: object(2)
memory usage: 87.2+ KB


55

In [35]:
def SMS_SPAM_DETECT_LSTM(input_shape, word_to_vec_map, word_to_index):
    """
    Function creating the Emojify-v2 model's graph.

    Arguments:
    input_shape -- shape of the input, usually (max_len,)
    word_to_vec_map -- dictionary mapping every word in a vocabulary into its 50-dimensional vector representation
    word_to_index -- dictionary mapping from words to their indices in the vocabulary (400,001 words)

    Returns:
    model -- a model instance in Keras
    """

    ### START CODE HERE ###
    # Define sentence_indices as the input of the graph.
    # It should be of shape input_shape and dtype 'int32' (as it contains indices, which are integers).
    sentence_indices = Input(shape=(input_shape), dtype='int32')

    # Create the embedding layer pretrained with GloVe Vectors (≈1 line)
    embedding_layer = pretrained_embedding_layer(word_to_vec_map, word_to_index)

    # Propagate sentence_indices through your embedding layer
    # (See additional hints in the instructions).
    embeddings = embedding_layer(sentence_indices)

    # Propagate the embeddings through an LSTM layer with 128-dimensional hidden state
    # The returned output should be a batch of sequences.
    X = LSTM(units = 128, return_sequences= True)(embeddings)
    # Add dropout with a probability of 0.5
    X = Dropout(rate = 0.5 )(X)
    # Propagate X trough another LSTM layer with 128-dimensional hidden state
    # The returned output should be a single hidden state, not a batch of sequences.
    X = LSTM(units = 128, return_sequences= False)(X)
    # Add dropout with a probability of 0.5
    X = Dropout(rate = 0.5 )(X)
    # Propagate X through a Dense layer with 2 units
    X = Dense(units = 2)(X)
    # Add a softmax activation
    X = Activation('softmax')(X)

    # Create Model instance which converts sentence_indices into X.
    model = Model(inputs=sentence_indices, outputs=X)

    ### END CODE HERE ###

    return model

In [39]:
model = SMS_SPAM_DETECT_LSTM((maxLen,), word_to_vec_map, word_to_index)
model.summary()

Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_2 (InputLayer)        [(None, 55)]              0         
                                                                 
 embedding_1 (Embedding)     (None, 55, 50)            20000050  
                                                                 
 lstm (LSTM)                 (None, 55, 128)           91648     
                                                                 
 dropout (Dropout)           (None, 55, 128)           0         
                                                                 
 lstm_1 (LSTM)               (None, 128)               131584    
                                                                 
 dropout_1 (Dropout)         (None, 128)               0         
                                                                 
 dense (Dense)               (None, 2)                 258   

In [42]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [43]:
X_train_np = np.array(X_train)
X_train_indices = sentences_to_indices(X_train_np, word_to_index, maxLen)
Y_train_np = np.array(Y_train)
label_encoder = LabelEncoder()
Y_train_vec = label_encoder.fit_transform(Y_train_np)
Y_train_oh = to_categorical(Y_train_vec)

model.fit(X_train_indices, Y_train_oh, epochs = 50, batch_size = 32, shuffle=True)

Epoch 1/50
3/3 [==============================] - 7s 253ms/step - loss: 0.5999 - accuracy: 0.7000
Epoch 2/50
3/3 [==============================] - 1s 250ms/step - loss: 0.4898 - accuracy: 0.8250
Epoch 3/50
3/3 [==============================] - 1s 258ms/step - loss: 0.4745 - accuracy: 0.8250
Epoch 4/50
3/3 [==============================] - 1s 206ms/step - loss: 0.4595 - accuracy: 0.8250
Epoch 5/50
3/3 [==============================] - 1s 277ms/step - loss: 0.4555 - accuracy: 0.8250
Epoch 6/50
3/3 [==============================] - 1s 276ms/step - loss: 0.4528 - accuracy: 0.8250
Epoch 7/50
3/3 [==============================] - 1s 231ms/step - loss: 0.4954 - accuracy: 0.8250
Epoch 8/50
3/3 [==============================] - 1s 220ms/step - loss: 0.4759 - accuracy: 0.8250
Epoch 9/50
3/3 [==============================] - 1s 209ms/step - loss: 0.4725 - accuracy: 0.8250
Epoch 10/50
3/3 [==============================] - 1s 343ms/step - loss: 0.4985 - accuracy: 0.8125
Epoch 11/50
3/3 [==

In [ ]:
X_test_np = np.array(X_test)
Y_test_np = np.array(Y_test)
X_test_indices = sentences_to_indices(X_test_np, word_to_index, max_len = maxLen)
label_encoder = LabelEncoder()
#     print('Y before encoding',Y)
Y_test_vec = label_encoder.fit_transform(Y_test_np)
#     print('Y after encoding', Y)
#     print('Y_vec',Y_vec)
Y_test_oh = to_categorical(Y_test_vec)
loss, acc = model.evaluate(X_test_indices, Y_test_oh)
print()
print("Test accuracy = ", acc)

In [ ]:
label_encoder.classes_
# Get the classes (original labels) from the LabelEncoder
classes_dict = {index: label for index, label in enumerate(label_encoder.classes_)}
classes_dict

In [ ]:
# This code allows you to see the mislabelled examples
pred = model.predict(X_test_indices)
count = 0
for i in range(len(X_test_np)):
    num = np.argmax(pred[i])
#     if(classes_dict[num] == 'spam'):
#         print('predict spam')
#         # This model never predicts spam!!
#     if(classes_dict[num] == Y_test_np[i] and classes_dict[num] == 'spam'):
    if(classes_dict[num] != Y_test_np[i]):
        print('Mislabeled input:', X_test_np[i])
        count += 1
        print('EXPECT:'+Y_test_np[i]+' PREDICT:'+classes_dict[num])
        print()
        # 03/06/24: I think all of the spam got mislabeled!
print('Total count:',count)
#         print('Expected: ')

In [ ]:
# Change the sentence below to see your prediction. Make sure all the words are in the Glove embeddings.
x_test = np.array(['*((*(AMDDD 1800 NOW)))'])
X_test_indices_input = sentences_to_indices(x_test, word_to_index, maxLen)
print(x_test[0] +' '+  classes_dict[np.argmax(model.predict(X_test_indices_input))])